# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [7]:
# # !pip install openai
# !pip install autogen
# !pip install python-dotenv

In [8]:
from utils_ import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [13]:
# reset all those keys
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [14]:
from autogen import ConversableAgent

agent = ConversableAgent(
    # api_key = OPENAI_API_KEY,
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [17]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)


Sure! Here's a joke for you: Why don't scientists trust atoms? Because they make up everything!


In [16]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)
# no memory
# new state
# so use multiple agents

Sure, I'd be happy to! Just let me know which joke you'd like me to repeat.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [18]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# set up the environment

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [19]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy! Speaking of awards, I recently won the procrastination award. It was supposed to be given to me last year, but I finally got it last month.

--------------------------------------------------------------------------------
cathy (to joe):

That's hilarious, Joe! Sounds like you truly earned that award with impeccable timing. Remember, why put off for tomorrow what you can put off for the day after tomorrow, right?

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [20]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Hey Joe! Why did the scarecrow win an award? Because he was '
             'outstanding in his field!',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'Haha, Cathy! Speaking of awards, I recently won the '
             'procrastination award. It was supposed to be given to me last '
             'year, but I finally got it last month.',
  'name': 'joe',
  'role': 'assistant'},
 {'content': "That's hilarious, Joe! Sounds like you truly earned that award "
             'with impeccable timing. Remember, why put off for tomorrow what '
             'you can put off for the day after tomorrow, right?',
  'name': 'cathy',
  'role': 'user'}]


In [23]:
pprint.pprint(chat_result.cost)
# tokens ... cost

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 93,
                                                             'cost': 0.00025,
                                                             'prompt_tokens': 221,
                                                             'total_tokens': 314},
                                      'total_cost': 0.00025},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 93,
                                                             'cost': 0.00025,
                                                             'prompt_tokens': 221,
                                                             'total_tokens': 314},
                                      'total_cost': 0.00025}}


In [22]:
pprint.pprint(chat_result.summary)

("That's hilarious, Joe! Sounds like you truly earned that award with "
 'impeccable timing. Remember, why put off for tomorrow what you can put off '
 'for the day after tomorrow, right?')


## Get a better summary of the conversation

In [24]:
chat_result = joe.initiate_chat(
    cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy! Speaking of awards, I recently won the procrastination award. It was supposed to be given to me last year, but I finally got it last month.

--------------------------------------------------------------------------------
cathy (to joe):

That's hilarious, Joe! Sounds like you truly earned that award with impeccable timing. Remember, why put off for tomorrow what you can put off for the day after tomorrow, right?

--------------------------------------------------------------------------------


In [25]:
pprint.pprint(chat_result.summary)

('Joe and Cathy exchanged jokes about winning awards, poking fun at '
 'procrastination and timing.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [26]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [27]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, glad you're up for some laughs. Alright, here's one for you: Why did the tomato turn red? Because it saw the salad dressing!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Here's one for you: Why don't scientists trust atoms? Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one, Joe! Science jokes are always electron... I mean, electrifying!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, nice one, Cathy! You really know how to spark some laughter!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, thanks Joe! Gotta keep t

In [28]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was your pun about keeping the comedic fire burning! It was quite electrifying!

--------------------------------------------------------------------------------
cathy (to joe):

That's right! Thanks for the reminder. Take care, Joe! Keep those jokes coming!

--------------------------------------------------------------------------------
joe (to cathy):

Thanks, Cathy! You take care too. Remember, laughter is the best medicine! See you next time!

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go

--------------------------------------------------------------------------------


In [29]:
# basically, the point is, as long as you set a termination phrase
# the two agents will keep chat until they run into the
# termination phrase.